# Imaginary quantum oscillators

In this problem, we imagine a collection of quantum systems, each of which is in an initial energtic state (the total energy of the combined system remains constant). By pairing up two of these systems, they can sometimes be caused to osciallate indefinitely between different energy levels. How so?

When to systems are brought in contact, the system in the lower state will "pull" the equivalent its own energy. At each step in time, such a transfer of energy takes place until both systems are on the same level. If that never happens, we have successfully set up a quantum oscialltor! (Any resemblance with real-world phenomena is purely accidential.)

Example: System 1 is in state 3, System 2 in state 5. Then the transitions are `(3,5) -> (6,2) -> (4,4)` (osciallation stops). However, if you were to start with `(2,3)`, you'd get `(2,3) -> (4,1) -> (3,2)` and so on ad infinitum.

Given a (not necessarily even) number of such systems, `n`, with their assocuiated initial energy, what's the minimum number of systems that cannot be paired to up as a quantum oscillator?

# Solution

The first question, of course, is how we can decide whether two systems will turn into an osciallator when coupled. The idea here is to start form the end: Given a final state (both system have the same energy), what directions in energy-space can we move into? At first, only one move is possible: `(n, n) -> (n/2, 3/2 n)`. At the next stage, two move are possible, provided that n is even: `(n/4, 7/4 n)` and `(5/4n, 3/4n)` and so on until n can no longer be divided by a higher power of 2. You can convinced yourself that two systems will not match if their sum is a power of two (after dividing both by the greatest common denominator). Thus, (11,5) will reach a stable state, as will (9,3), but (11,2) will not.

Once we figured out how to do this for two systems, why not for all of them? The easiest way to store this information is in a upper-triangular matrix: Since `isMatch()` has to be symmetric in its elements and always returns 0 if you input the same element twice, it's enough to focus on those elements `m_i,j` where `i>j`. There's a small trick included here: If the number of systems is odd, there will always be one unpaired system. This is taken care of be extending the length of the original input list by one element, 0, and we require that `isMatch(0,x) = 1`. When set up this way, later on we can proceed "as if" we were dealing with an even number of systems.

The most complicated step, no that we've stored away all those matching results in a matrix is to figure out how many different pairs can be formed? Another way of framing this question is to ask which partitions of length 2 are possible for an (even-numbered) set. There are `(n-1)*(n-3), n > 3` of such partitions, indicating that we have to loop over two different collections. Here, the idea we pursue is the following:

- create a list containing number from 0 to the size of the set and store it inside a list: `[[0,1,2,3,4,5]]`. The numbers will later be used to retrieve specific elements of the matching matrix.
- start by fixing the first element of that list (0) and put each of the remaining numbers in the next place, then add to these new lists whatever remains from the original list: `[[0,1,2,3,4,5],[0,2,1,3,4,5],...,[0,5,1,2,3,4]]`. 
- now move on to the third element: For each list in our updated nested list, keep that element fixed, then proceed as in the step above. For a concrete case, the list `[0,2,1,3,4,5]` will be transformed into `[[0,2,1,3,4,5],[0,2,1,4,3,5], [0,2,1,5,3,4]`.

Next, we want to calculate exactly how many systems are oscillating for a given partition, a job that is done by `getSumOfPartition`. Here, we retrieve the values in the matching matrix `M` that correspond to the pairs reprented by our lists from above: `[0,2,1,3,4,5]` will go fetch the elements `M[0,2], M[1,3] and M[4,5]`.

The final step then consists in looping over all possible partitions and checking whether the result corresponds to the best possible case (matching all of them in case of an even number of systems, or one less then the total cound if that number is odd). If not, check if it's better than the current best value (and if so, update it). There's a bit of complications arising from having to consider odd/even cases, but apart from that the last part is quite straightforward.

In [60]:
import numpy as np

def gcd(x, y):    #greatest common denominator
    while(y): 
        x, y = y, x % y
    return x

def isPowerOfTwo(x):
    return x and (not(x & (x - 1)))  #use binary comparison

def isMatch(x,y): 
    x,y = max(x,y),min(x,y)
    if y == 0:
        return 1
    if x==y or isPowerOfTwo((x+y)//gcd(x,y)):
        return 0
    else:
        return 2

def matchingMatrix(systems):
    length = len(systems)    
    arr = np.array([[0]*length]*length) # create empty array of size n x n
    for i in range(0,length):
        for j in range(i+1,length):
            arr[i][j] = isMatch(systems[i], systems[j])
    return arr

def findPartitions(systems):
    indices = [list(range(len(systems)))]
    for i in range(0, len(l)-2, 2):
        indices = [el[:i]+[el[i],el[j]]+el[i+1:j]+el[j+1:] for el in nested_list for j in range(i+1, len(el))]
    return indices

def getSumOfPartition(matrix, partition):
    pairs = [(partition[i],partition[i+1]) for i in range(0,len(partition), 2)]
    return sum([matrix[p] for p in pairs])
    
def solution(systems):
    
    is_odd = len(systems)%2
    
    if is_odd==1:
        systems += [0]
    
    matching_mat = matchingMatrix(systems)
    partitions = findPartitions(systems)
    best_result = len(systems)-is_odd
    res = 0
    
    for p in partitions:
        matches = getSumOfPartition(matching_mat, p)
        if matches == best_result:
            return is_odd
        if matches > res:
            res = matches
    return best_result - res + is_odd


In [63]:
# Example
solution([1, 7, 1, 21, 13, 19])

0